In [1]:
# Imports all the necessary libraries

from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
!mkdir test_repo

repo_path = "test_repo/"
Repo.clone_from("https://github.com/VivekJhaConnect/Time-Series-Analysis.git", to_path=repo_path)

<git.repo.base.Repo 'c:\\Users\\ADMIN\\Documents\\GitHub\\Generative-AI-Projects\\Source-Code-Analysis\\research\\test_repo\\.git'>

In [6]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob="**/*",
                                       suffixes=[".ipynb", ".py"],
                                       parser=LanguageParser(language=Language.PYTHON, parser_threshold=0.5))

In [7]:
documents = loader.load()

In [8]:
# documents

In [19]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=300,
    chunk_overlap=20
)

In [20]:
texts = documents_splitter.split_documents(documents)

In [21]:
# texts

In [22]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=r"C:\Users\ADMIN\Documents\venv\.env")
os.environ['PINECONE_API_KEY']=os.getenv("PINECONE_API_KEY")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [23]:
enbeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [24]:
vectordb = Chroma.from_documents(texts, embedding=enbeddings, persist_directory="chroma_db")

In [25]:
vectordb.persist()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12136\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [26]:
llm = ChatOpenAI()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12136\44520985.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


In [27]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12136\1764516168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)


In [28]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type='mmr', search_kwargs={'n': 8}), memory=memory)

In [31]:
question = "where is the Seasonal Autoregressive Integrated Moving Average code?"

In [32]:
result = qa(question)
print(result['answer'])

The Seasonal Autoregressive Integrated Moving Average (SARIMA) code is located in the output of the cell that mentions "#### Auto ARIMA." The SARIMA model is often implemented using libraries like pmdarima or statsmodels.
